In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from models.mida import Mida
from models.mice import Mice

import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from fancyimpute import KNN, SoftImpute

from preprocessing.data_loading import DataSet
from preprocessing.missing_value_generation import mcar_generator, mar_generator, mnar_generator
from models.imputation_wrapper import SingleImputationWrapper, MultiImputationWrapper
from analysis.evaluation import ClassificationEvaluation, LnormEvaluation, TimeEvaluation

In [10]:
dataset = DataSet('ionosphere', 'class')
amputations = (mcar_generator(), mar_generator(2), mnar_generator(2))

In [11]:
clf_evaluation = ClassificationEvaluation(KNeighborsClassifier(5), dataset)
lnorm_evaluation = LnormEvaluation(dataset)
time_evaluation = TimeEvaluation(dataset)

In [12]:
# result on complete dataset
clf_evaluation.evaluate_result([dataset.complete_data()], 0, 'None', 'None')
lnorm_evaluation.evaluate_result([dataset.complete_data()], 0, 'None', 'None')

In [13]:
imputations = [SingleImputationWrapper(KNN(3, verbose=False)), SingleImputationWrapper(SoftImpute(verbose=False)),
               MultiImputationWrapper(Mice(40, verbose=False)), MultiImputationWrapper(Mida(40, verbose=False))]

In [14]:
# different methods
for amputation in amputations:
    dataset.init_missing_data(amputation)
    
    # increasing rate p
    for p in [0.2, 0.4, 0.6, 0.8]:
        dataset.ampute_values(p)
        
        # apply each imputation
        for imputation in imputations:
            results, exec_time = imputation.complete(dataset.missing_data())
            clf_evaluation.evaluate_result(results, p, amputation.name(), imputation.name())
            lnorm_evaluation.evaluate_result(results, p, amputation.name(), imputation.name())
            time_evaluation.evaluate_result(exec_time, p, amputation.name(), imputation.name(), imputation.number())
            print(amputation.name() + ': ' + str(p) + ' ' + imputation.name() + ' done')

/home/marcus/Studium/Master/Semester_1/Smart Representation/Imputation/preprocessing/missing_value_generation.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result[c][rows] = np.nan


mcar: 0.2 KNN done


/home/marcus/anaconda3/lib/python3.6/site-packages/fancyimpute-0.3.1-py3.6.egg/fancyimpute/soft_impute.py:100: RuntimeWarning: divide by zero encountered in double_scalars


mcar: 0.2 SoftImpute done
mcar: 0.2 Mice done
mcar: 0.2 Mida done
mcar: 0.4 KNN done
mcar: 0.4 SoftImpute done
mcar: 0.4 Mice done
mcar: 0.4 Mida done
mcar: 0.6 KNN done
mcar: 0.6 SoftImpute done
mcar: 0.6 Mice done
mcar: 0.6 Mida done
mcar: 0.8 KNN done
mcar: 0.8 SoftImpute done
mcar: 0.8 Mice done
mcar: 0.8 Mida done


/home/marcus/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:2248: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/home/marcus/Studium/Master/Semester_1/Smart Representation/Imputation/preprocessing/missing_value_generation.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result[c][rows] = np.nan


mar: 0.2 KNN done
mar: 0.2 SoftImpute done
mar: 0.2 Mice done
mar: 0.2 Mida done
mar: 0.4 KNN done
mar: 0.4 SoftImpute done
mar: 0.4 Mice done
mar: 0.4 Mida done
mar: 0.6 KNN done
mar: 0.6 SoftImpute done
mar: 0.6 Mice done
mar: 0.6 Mida done
mar: 0.8 KNN done
mar: 0.8 SoftImpute done
mar: 0.8 Mice done
mar: 0.8 Mida done


/home/marcus/Studium/Master/Semester_1/Smart Representation/Imputation/preprocessing/missing_value_generation.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result[c][rows] = np.nan


mnar: 0.2 KNN done
mnar: 0.2 SoftImpute done
mnar: 0.2 Mice done
mnar: 0.2 Mida done
mnar: 0.4 KNN done
mnar: 0.4 SoftImpute done
mnar: 0.4 Mice done
mnar: 0.4 Mida done
mnar: 0.6 KNN done
mnar: 0.6 SoftImpute done
mnar: 0.6 Mice done
mnar: 0.6 Mida done
mnar: 0.8 KNN done
mnar: 0.8 SoftImpute done
mnar: 0.8 Mice done
mnar: 0.8 Mida done


In [15]:
clf_evaluation.evaluation_results
clf_evaluation.dump_results()

In [16]:
lnorm_evaluation.evaluation_results
lnorm_evaluation.dump_results()

In [17]:
time_evaluation.evaluation_results
time_evaluation.dump_results()